# 📱 SMS Smishing Annotation Tool - Compact Unified

Công cụ gán nhãn tương tác để phân loại tin nhắn SMS smishing và metadata - **TẤT CẢ TRONG MỘT GIAO DIỆN!**

## ✨ Tính năng chính:

### 🎯 **Giao diện thống nhất tinh gọn**
- Gán nhãn **label** và chỉnh sửa **metadata** cùng lúc trên một màn hình
- Hiển thị chỉ tin nhắn chưa được gán nhãn
- Metadata được điền trước, sau đó click nút label để lưu tất cả cùng lúc
- Tiết kiệm thời gian, không cần chuyển đổi giữa các mode

### 📝 **Chức năng annotation:**
- **Label**: Gán nhãn `smishing` (1) hoặc `not smishing` (0)
- **Metadata**:
  - 🔗 `has_url`: Có chứa URL hay không
  - 📞 `has_phone_number`: Có chứa số điện thoại hay không
  - 📱 `sender_type`: Loại người gửi (personal_number, shortcode, brandname, unknown)

### 🛠️ **Công cụ hỗ trợ:**
- ↩️ **Undo**: Hoàn tác thao tác gần nhất
- ⏭️ **Skip**: Bỏ qua tin nhắn, chuyển sang tin kế tiếp
- ⬅️ **Prev**: Quay lại tin nhắn trước
- 🔢 **Jump**: Nhảy tới tin nhắn cụ thể theo index
- 💾 **Auto-save**: Tự động lưu vào file CSV sau mỗi thay đổi

---

## 📋 Cấu trúc dữ liệu:

| Column | Type | Description |
|--------|------|-------------|
| `content` | string | Nội dung tin nhắn SMS |
| `label` | int | Nhãn smishing: `0` = Not smishing, `1` = smishing |
| `has_url` | int | Có URL: `0` = Không, `1` = Có |
| `has_phone_number` | int | Có SĐT: `0` = Không, `1` = Có |
| `sender_type` | string | Loại người gửi: `personal_number`, `shortcode`, `brandname`, `unknown` |

---

## 🚀 Bắt đầu:

1. **Chạy Cell 1**: Import thư viện
2. **Chạy Cell 2**: Load dữ liệu
3. **Chạy Cell 3**: Khởi tạo class Annotation Tool
4. **Chạy Cell 4**: Hiển thị giao diện annotation
5. **Bắt đầu gán nhãn!** 🎉

In [2]:
# Import necessary libraries
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully!")


✓ Libraries imported successfully!


In [5]:
# Load data
DATA_FILE = '../../dataset/dataset.csv'     #thay địa chỉ tương đối vào đây

def load_data():
    """Load the SMS data from CSV file."""
    df = pd.read_csv(DATA_FILE)
    return df

def save_data(df):
    """Save the SMS data to CSV file."""
    df.to_csv(DATA_FILE, index=False)
    
# Load initial data
df = load_data()
print(f"✓ Data loaded: {len(df)} total messages")
print(f"  - Labeled: {df['label'].notna().sum()}")
print(f"  - Unlabeled: {df['label'].isna().sum()}")


✓ Data loaded: 2603 total messages
  - Labeled: 2603
  - Unlabeled: 0


In [ ]:
# Annotation Tool Class - RELABELING VERSION
class SMSAnnotationTool:
    def __init__(self, df):
        self.df = df
        # CHANGED: Use all indices so we can navigate through already labeled data
        self.indices = df.index.tolist() 
        self.current_idx = 0
        self.history = []
        
        self.create_widgets()
        
    def create_widgets(self):
        button_layout = widgets.Layout(width='140px', height='40px')
        
        self.progress_label = widgets.HTML(value=self.get_progress_html())
        self.nav_label = widgets.HTML(value=self.get_nav_html())
        self.message_area = widgets.HTML(
            value=self.get_message_html(),
            layout=widgets.Layout(width='100%', border='2px solid #ddd', padding='15px', margin='10px 0')
        )
        
        # LABEL BUTTONS
        self.not_smishing_btn = widgets.Button(description='✅ Not smishing (0)', button_style='success', layout=button_layout)
        self.smishing_btn = widgets.Button(description='🚫 smishing (1)', button_style='danger', layout=button_layout)
        
        # METADATA WIDGETS
        self.has_url_checkbox = widgets.Checkbox(value=False, description='🔗 URL', indent=False)
        self.has_phone_checkbox = widgets.Checkbox(value=False, description='📞 Phone', indent=False)
        self.sender_dropdown = widgets.Dropdown(
            options=['', 'personal_number', 'shortcode', 'brandname', 'unknown'],
            value='', description='📱 Sender:', layout=widgets.Layout(width='250px')
        )
        
        # NAVIGATION
        self.prev_btn = widgets.Button(description='⬅️ Prev', button_style='info', layout=button_layout)
        self.next_btn = widgets.Button(description='Next ➡️', button_style='info', layout=button_layout)
        self.undo_btn = widgets.Button(description='↩️ Undo', button_style='warning', layout=button_layout)
        
        self.jump_input = widgets.IntText(value=0, description='Jump:', layout=widgets.Layout(width='130px'))
        self.jump_btn = widgets.Button(description='Go', button_style='info', layout=widgets.Layout(width='60px', height='40px'))
        
        # Bind events
        self.not_smishing_btn.on_click(lambda b: self.annotate(0))
        self.smishing_btn.on_click(lambda b: self.annotate(1))
        self.next_btn.on_click(lambda b: self.move_next())
        self.prev_btn.on_click(lambda b: self.previous())
        self.undo_btn.on_click(lambda b: self.undo())
        self.jump_btn.on_click(lambda b: self.jump_to())
        
        self.output = widgets.Output()

    def get_progress_html(self):
        total = len(self.df)
        labeled = self.df['label'].notna().sum()
        percentage = (labeled / total * 100) if total > 0 else 0
        return f"<h3>📊 Total Labeled: {labeled}/{total} ({percentage:.1f}%)</h3>"

    def get_message_html(self):
        idx = self.indices[self.current_idx]
        row = self.df.loc[idx]
        label_val = row.get('label')
        
        # Color coding based on current label
        color = "#4CAF50" if label_val == 0 else "#f44336" if label_val == 1 else "#ff9800"
        text = "✅ Not smishing" if label_val == 0 else "🚫 smishing" if label_val == 1 else "⚠️ Unlabeled"
        
        content = str(row['content']).replace('<', '&lt;').replace('>', '&gt;')
        return f"<div style='border-left: 5px solid {color}; padding: 10px;'><b>{text}</b> (Idx: {idx})<br><br>{content}</div>"

    def get_nav_html(self):
        return f"<b>Item:</b> {self.current_idx + 1} / {len(self.indices)}"

    def update_display(self):
        self.progress_label.value = self.get_progress_html()
        self.nav_label.value = self.get_nav_html()
        self.message_area.value = self.get_message_html()
        
        idx = self.indices[self.current_idx]
        row = self.df.loc[idx]
        self.has_url_checkbox.value = bool(row.get('has_url') == 1)
        self.has_phone_checkbox.value = bool(row.get('has_phone_number') == 1)
        self.sender_dropdown.value = str(row.get('sender_type')) if pd.notna(row.get('sender_type')) else ''
        self.prev_btn.disabled = self.current_idx == 0

    def annotate(self, label):
        idx = self.indices[self.current_idx]
        self.df.loc[idx, 'label'] = label
        self.df.loc[idx, 'has_url'] = 1 if self.has_url_checkbox.value else 0
        self.df.loc[idx, 'has_phone_number'] = 1 if self.has_phone_checkbox.value else 0
        self.df.loc[idx, 'sender_type'] = self.sender_dropdown.value if self.sender_dropdown.value else None
        
        save_data(self.df) # Will trigger PermissionError if file is open!
        
        if self.current_idx < len(self.indices) - 1:
            self.current_idx += 1
        self.update_display()

    def move_next(self):
        if self.current_idx < len(self.indices) - 1:
            self.current_idx += 1
            self.update_display()

    def previous(self):
        if self.current_idx > 0:
            self.current_idx -= 1
            self.update_display()

    def jump_to(self):
        target = self.jump_input.value
        if target in self.df.index:
            self.current_idx = self.df.index.get_loc(target)
            self.update_display()

    def undo(self):
        # Implementation of undo omitted for brevity, but can follow your original logic
        pass

    def display(self):
        ui = widgets.VBox([
            self.progress_label, self.nav_label, self.message_area,
            widgets.HBox([self.has_url_checkbox, self.has_phone_checkbox, self.sender_dropdown]),
            widgets.HBox([self.not_smishing_btn, self.smishing_btn]),
            widgets.HBox([self.prev_btn, self.next_btn]),
            widgets.HBox([self.jump_input, self.jump_btn]),
            self.output
        ])
        display(ui)
        self.update_display()

✓ Compact Unified Annotation Tool Created!


## 🎯 Hướng dẫn sử dụng

Chạy cell dưới để khởi động công cụ annotation với **giao diện tinh gọn tất cả trong một**!

---

### 📋 **QUY TRÌNH GÁN NHÃN**

Giao diện hiển thị tất cả chức năng trên cùng một màn hình:

#### **Bước 1: Đọc tin nhắn** 📖
- Đọc nội dung tin nhắn hiển thị
- Xem trạng thái label hiện tại (nếu có)
- Kiểm tra metadata hiện có (🔗 URL, 📞 Phone, 📱 Sender)

#### **Bước 2: Chỉnh sửa metadata** 📝
Ngay phía dưới tin nhắn, điều chỉnh các trường metadata:
- **🔗 URL**: Tick nếu tin nhắn có chứa link/URL
- **📞 Phone**: Tick nếu có số điện thoại
- **📱 Sender**: Chọn loại người gửi
  - `personal_number`: Số cá nhân (10-11 số: 0912345678)
  - `shortcode`: Mã ngắn (4-6 số: 8077, 9029)
  - `brandname`: Tên thương hiệu (MoMo, Viettel)
  - `unknown`: Không xác định

#### **Bước 3: Gán nhãn & lưu** 🏷️
Click một trong hai nút:
- **✅ Not smishing (0)**: Tin nhắn hợp pháp
- **🚫 smishing (1)**: Tin nhắn lừa đảo

➡️ **Metadata + Label sẽ được lưu cùng lúc!**

---

### 🧭 **CÁC NÚT ĐIỀU HƯỚNG**

| Nút | Chức năng |
|-----|-----------|
| **⬅️ Prev** | Xem lại tin nhắn trước đó |
| **⏭️ Skip** | Bỏ qua tin nhắn này, chuyển sang tin kế tiếp |
| **↩️ Undo** | Hoàn tác thao tác vừa rồi |
| **Jump + Go** | Nhảy tới tin nhắn cụ thể theo index |

---

### 💡 **MẸO SỬ DỤNG**

✅ **Hiệu quả:**
- Điền metadata trước khi click nút label → tiết kiệm thời gian
- Dùng **Skip** khi không chắc chắn, quay lại sau
- **Undo** nếu nhầm lẫn ngay sau khi lưu

✅ **An toàn:**
- Mọi thao tác **tự động lưu ngay lập tức**
- Đóng notebook bất cứ lúc nào, dữ liệu được giữ nguyên
- File lưu tại: `data/sms_data.csv`

✅ **Theo dõi:**
- Progress bar hiển thị % hoàn thành
- Counter cho biết đang ở tin nhắn nào / tổng số unlabeled

---

**Chạy cell bên dưới để bắt đầu!**

In [10]:
# Initialize and display the annotation tool
tool = SMSAnnotationTool(df)
tool.display()


AttributeError: 'SMSAnnotationTool' object has no attribute 'unlabeled_indices'

## 🛠️ Công cụ bổ sung

Sử dụng các cell dưới đây cho các tác vụ hỗ trợ:

### 📊 Cell 7: Kiểm tra thống kê hiện tại
Xem tổng quan tiến độ gán nhãn và phân bố dữ liệu

### 🔄 Cell 8: Reload dữ liệu
Tải lại dữ liệu từ file (hữu ích khi có thay đổi từ bên ngoài)

### 💾 Cell 9: Export dữ liệu đã gán nhãn
Xuất ra file riêng chỉ chứa các tin nhắn đã được gán nhãn

In [15]:
# Check current statistics
df_current = load_data()
total = len(df_current)
labeled = df_current['label'].notna().sum()
unlabeled = total - labeled
smishing_count = (df_current['label'] == 1).sum()
not_smishing_count = (df_current['label'] == 0).sum()

print("=" * 50)
print("CURRENT STATISTICS")
print("=" * 50)
print(f"Total messages:     {total}")
print(f"Labeled:            {labeled} ({labeled/total*100:.1f}%)")
print(f"  - smishing (1):       {smishing_count}")
print(f"  - Not smishing (0):   {not_smishing_count}")
print(f"Unlabeled:          {unlabeled} ({unlabeled/total*100:.1f}%)")
print("=" * 50)


CURRENT STATISTICS
Total messages:     245
Labeled:            245 (100.0%)
  - smishing (1):       221
  - Not smishing (0):   24
Unlabeled:          0 (0.0%)


In [16]:
# Reload data and restart tool (if needed)
# Run this cell if you want to restart the annotation tool with fresh data

df = load_data()
print(f"Data reloaded: {len(df)} messages")
print(f"Unlabeled: {df['label'].isna().sum()}")
print("\nTo start annotating, create a new tool instance:")
print("tool = SMSAnnotationTool(df)")
print("tool.display()")


Data reloaded: 245 messages
Unlabeled: 0

To start annotating, create a new tool instance:
tool = SMSAnnotationTool(df)
tool.display()


In [14]:
# Export labeled data only (optional)
# Uncomment and run to export only the labeled messages

# df_labeled = df_current[df_current['label'].notna()]
# df_labeled.to_csv('data/labeled_sms_only.csv', index=False)
# print(f"Exported {len(df_labeled)} labeled messages to data/labeled_sms_only.csv")
